**MySQL - pruebas con los dos .csv de 36/40 registros. Ver bbdd_musicstream_test.sql en Repositorio**

In [ ]:
## archivos de los tests todos tienen cc- por delante (sobretodo para que separaran)
# he intentado comentar cosas que suregen

In [1]:
import mysql.connector
from mysql.connector import errorcode

In [2]:
import pandas as pd

In [3]:
# conectar con mysql -- cambiar PASSWORD! ;)
cnx = mysql.connector.connect(user='root', password='Hist0ryF0r3v3r',
                              host='127.0.0.1')

cursor = cnx.cursor()

In [4]:
# conectar con bbdd
cursor.execute("USE bbdd_musicstream_team_2")    

In [5]:
# averiguar conexion con bbdd
cnx.database                                    

'bbdd_musicstream_team_2'

In [6]:
# importar .csv para tener un DataFrame
df_lastfm = pd.read_csv('cc-artistas_lastfm_test_36_bios_limpieza_test.csv')
df_spotify = pd.read_csv('canciones_spotify_test_40.csv')

In [7]:
# csv tenia fecha USA MM/DD/YYYY esto convierte a YYYY-MM-DD para que funcione en MySQL
df_spotify['release_date'] = (
    pd.to_datetime(df_spotify['release_date'], errors='coerce')
      .dt.strftime('%Y-%m-%d')
)

In [8]:
df_spotify.head()

,id,track_name,artist_name,year,genre,album_type,release_date,popularity
0,1WHNqqRWhJVZIdCScFKtl5,Washington on Your Side,Leslie Odom Jr.,2015,soundtrack,album,2015-09-25,67
1,1llkMtLL4vbe0pr3NV2ckg,Let Me Hear,"Fear, and Loathing in Las Vegas",2015,soundtrack,album,2015-09-30,58
2,39IOkz6LpC1qc5Wnt0T07r,Dance of the Druids (feat. Raya Yarbrough),Bear McCreary,2015,soundtrack,album,2015-02-10,53
3,5L9MJsGqzTRD09rSzHkCDy,Home,Toby Fox,2015,soundtrack,album,2015-09-15,62
4,0LpHC9mhPAQC98IjXZIrif,The Election of 1800,Daveed Diggs,2015,soundtrack,album,2015-09-25,66


In [9]:
df_lastfm.head()

,artist_name,biography,listeners,playcount,similar_artists,bio_summary_txt,bio_content_txt
0,Leslie Odom Jr.,"<a href=""https://www.last.fm/music/Leslie+Odo...",780378,33296954,"['Christopher Jackson', 'Phillipa Soo', 'Renée...","Leslie Odom, Jr (born August 6, 1981) is an Am...","Leslie Odom, Jr (born August 6, 1981) is an Am..."
1,"Fear, and Loathing in Las Vegas","Fear, and Loathing in Las Vegas, es una banda ...",213493,7951958,"['a crowd of rebellion', 'MY FIRST STORY', 'co...","Fear, and Loathing in Las Vegas is a electroni...","Fear, and Loathing in Las Vegas is a electroni..."
2,Bear McCreary,Nacido el 17 de febrero de 1979 en Fort Lauder...,520843,20700177,"['Richard Gibbs', 'Lorne Balfe', 'Howard Shore...","Bear McCreary (born February 17, 1979) is an A...","Bear McCreary (born February 17, 1979) is an A..."
3,Toby Fox,"Toby Fox (Boston, Massachussetts, 11 de octubr...",1106419,133289863,"['Lena Raine', 'Omori', 'Laura Shigihara', 'Ni...",Toby Fox is an American developer and composer...,Toby Fox is an American developer and composer...
4,Daveed Diggs,"<a href=""https://www.last.fm/music/Daveed+Dig...",395029,4603573,"['Christopher Jackson', 'Phillipa Soo', 'Lesli...","Daveed Daniele Diggs (born January 24, 1982 in...","Daveed Daniele Diggs (born January 24, 1982 in..."


**NOTA - hay que hacer limpieza de datos**

In [10]:
# BIOGRAFIAS - muchas salen con link raro (serian muchos para dejar nulos)

## Opcion 1 ChatGPT - Reemplazar biografías que son solo links con None (valor nulo)
#df_lastfm['biography'] = df_lastfm['biography'].apply(
#    lambda x: None if isinstance(x, str) and x.strip().startswith('http') else x
#)

## Opcion 2 - 
# ver otro Jupyter "CC-python-pandas-artistas-biografia-limpeza-test.ipynb" y sus resultados "CC-artistas_lastfm_test_36_bios_limpieza_test.csv"
# mejor pero no del todo... hay que afinarlo.

In [11]:
# SIMILAR ARTISTS - MySQL probablemente lo vera como cadena no lista "['Stray Kids', 'ATEEZ', 'NCT Dream']"

## Opcion 1 ChatGPT - 
#import ast  # para convertir el string de lista a lista real

#def clean_similar(x):
#    if isinstance(x, str):
#        try:
#            lista = ast.literal_eval(x)  # convierte '["A","B"]' → ["A","B"]
#            if isinstance(lista, list):
#                return ', '.join(lista)
#        except (ValueError, SyntaxError):
#            pass
#    return x  # deja como está si no hay lista

# df_lastfm['similar_artists'] = df_lastfm['similar_artists'].apply(clean_similar)


In [12]:
# CARACTERES EXTRANOS - (Ã©, Â, ñ mal codificado) -- agregar al DataFrame -->, encoding=;utf-8'
# df_lastfm = pd.read_csv('artistas_lastfm_test_36.csv', encoding='utf-8')
# df_spotify = pd.read_csv('canciones_spotify_test_40.csv', encoding='utf-8')

In [13]:
#NULOS - convertir NULOS de NaN a None (para que luego SQL acepte la importacion de datos)
df_lastfm  = df_lastfm.where(pd.notnull(df_lastfm), None)
df_spotify = df_spotify.where(pd.notnull(df_spotify), None)

In [ ]:
# VALUES - insertar datos en lastfm (con variable DataFrame) | 2 bios extra para comparar outputs (a revisar/pulir mas)
sql_lastfm = """
INSERT INTO lastfm (artist_name, biography, listeners, playcount, similar_artists, bio_summary_txt, bio_content_txt)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(sql_lastfm, df_lastfm.values.tolist())

In [15]:
# VALUES - insertar datos en lastfm (con variable DataFrame)
sql_spotify = """
INSERT INTO spotify (id, track_name, artist_name, `year`, genre, album_type, release_date, popularity)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(sql_spotify, df_spotify.values.tolist())

In [ ]:
# COMMIT - guardar los valores en MySQL
cnx.commit()       

In [ ]:
# Cerrar la connexion con MySQL
cursor.close()
cnx.close()